In [ ]:
import openeo
import geopandas as gpd
import os
import rasterio
import plotly.express as px
import numpy as np
import pandas as pd

## How to use GeoPandas and OpenEo

This notebook shows some of the functions, libraries and file formats we use in the chapter, in case you're not familiar with them.

GeoPandas is a library to work with tables holding geographic information. This geographic information is represented by sets of coordinates, which can define points, lines or polygons (or multi-polygons).

Geopackage is a file type (`.gpkg`) that can hold layers of geographical information. In our case, the notebook in `/data_preparation` created a `.gpkg` with polygon/multipolygon geometries (Paris' parks) and a layer f points (Paris's Parks centroids).

We can explore the `.gpkg` file with GeoPandas and the `list_layers` method:

In [ ]:
geopackage_path = "../src/data/paris_parks.gpkg"

layer_names = gpd.list_layers(geopackage_path)

print(layer_names)

We can uise each layer to create a GeoPandas GeoDataFrame. GepDataFrames have tabular data (just like a pandas DataFrame), but also geographic information. Each column in the GeoDataFrame is an **attribute** of the geographical object (for example, `postal_code` is an attribute of Paris' parks):

In [ ]:
gdf_paris_parks = gpd.read_file(geopackage_path, layer="parks_polygons")
gdf_park_centroid = gpd.read_file(geopackage_path, layer="parks_centroids")

In [ ]:
gdf_paris_parks.head()

Just like pandas, GeoPandas has a `.plot()` method, it "knows" what to plot. `gdf_paris_parks` has polygons, and `gdf_park_centroid` has points:

In [ ]:
gdf_paris_parks.plot()

In [ ]:
gdf_park_centroid.plot()

## Ploting with Plotly

We can plot GeoDataFrames with Plotly.
Since we have centroids for each polygon, we can use them to center the map around them!

In [ ]:
def plot_park_with_centroid(
    gdf_parks, gdf_park_centroid, park_id, map_style="open-street-map"
):
    """
    Plots a specific park with its centroid on a Plotly map.

    Args:
        gdf_parks (geopandas.GeoDataFrame): GeoDataFrame containing park polygons.
        gdf_park_centroid (geopandas.GeoDataFrame): GeoDataFrame containing park centroids.
        park_id (str or int): The ID of the park to plot.
        map_style (str, optional): Mapbox style. Defaults to "open-street-map".

    Returns:
        plotly.graph_objects.Figure: A Plotly map figure, or None if the park is not found.
    """

    park_polygon = gdf_parks[gdf_parks["id"] == park_id].copy()
    park_centroid = gdf_park_centroid[gdf_park_centroid["id"] == park_id].copy()

    if park_polygon.empty or park_centroid.empty:
        return None

    # Get centroid coordinates
    centroid_point = park_centroid["geometry"].iloc[0]
    center = {"lat": centroid_point.y, "lon": centroid_point.x}

    fig = px.choropleth_mapbox(
        park_polygon,
        geojson=park_polygon.geometry.__geo_interface__,
        locations=park_polygon.index,
        center=center,
        mapbox_style=map_style,
        zoom=15,
        opacity=0.5,
        hover_name="name",
        hover_data={
            "name": True,
            "type": True,
            "category": True,
            "area_sqm": True,
        },
    )

    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    return fig

In [ ]:
plot_park_with_centroid(
    gdf_parks=gdf_paris_parks,
    gdf_park_centroid=gdf_park_centroid,
    park_id=2425,
    map_style="open-street-map",
)

## Converting to DataFrame

In case you need it, you can convert a GeoDataFrame into a DataFrame:

In [ ]:
df_paris_parks = pd.DataFrame(gdf_paris_parks)

In [ ]:
df_paris_parks.groupby("type").size().reset_index(name="count")

## OpenEO

[OpenEO](https://pypi.org/project/openeo/) ("EO" for Earth Observation) is a Python library that provides an API to interact with backends that serve geographic information.

For our app, we'll be using Sentinel-2 data, from the European Space Agency (ESA).

**You need to create an account: https://dataspace.copernicus.eu/**.

![](../img/account.png)

**You'll also need to create credentials**:

* Go to your account's settings (likely: https://shapps.dataspace.copernicus.eu/dashboard/#/account/settings)


### Important resources:

* [OpenEO's documentation about datacubes, the most important concept to understand the app's algorithms](https://openeo.org/documentation/1.0/datacubes.html#what-are-datacubes]).

* [Notebooks to chek out]( https://dataspace.copernicus.eu/cases).

## Defining functions

These are the functions we use in the pipelines, for pour Scenario and also for our Generic Data Nodes.

The first one retrieves a Polygon from our previou GeoDataFrame:

In [ ]:
def get_polygon(gdf_paris_parks, id):
    polygon = gdf_paris_parks[gdf_paris_parks["id"] == id].geometry.__geo_interface__
    return polygon

**IMPORTANT:** This function returns the connector to our Copernicus's space. You'll need to provide your credentials.

**We highly recommend using environment variables to store your credentials!**

In [ ]:
def connect_to_copernicus(
    client_id, client_secret, url="https://openeo.dataspace.copernicus.eu"
):
    connection = openeo.connect(url)
    connection.authenticate_oidc_client_credentials(
        client_id=client_id, client_secret=client_secret
    )
    return connection

A **datacube** holds the following dimensions:

* **Time**: we need to provide the time extent for the Earth's Observations. Sentinel 2 goes over each spot on Earth every 3 days more or less. Each observation is one picture from the satellite.
* **Spatial Extent**: The surface of the observed area. Bigger surfaces give more pixels.
* **Spectral resolution:** These are the "bands" that record specific waves. For example, Band 04 has Near Infra Red information, and Band 08 has red information.

In our case, **we'll use the app's UI to select a specific year for time obeservation, and a specific Paris' park (a polygon) for spatial extent**.

In [ ]:
def create_datacube(polygon, start, end, connection):
    datacube = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=[start, end],
        spatial_extent=polygon,
        bands=["B04", "B08", "SCL"],  # Red and NIR for NDVI and SCL for clouds
    )
    return datacube

The SCL band can be used to mask cloud data. When satellites take images of the Earth, sometimes... there are clouds. Cloud pixels alter the observation results, so we can mask them (we remove them from the observation, from the cube).

In [ ]:
def mask_clouds(datacube):
    """
    Applies a cloud mask to a given data cube by filtering out cloud-covered pixels.

    Args:
    datacube (openeo.DataCube): The data cube containing Sentinel-2 bands.

    Returns:
    openeo.DataCube: The masked data cube with clouds removed.
    """
    scl = datacube.band("SCL")
    cloud_mask = (scl == 8) | (scl == 9)  # Cloud classes
    datacube = datacube.mask(cloud_mask)
    return datacube

**We can create datacube from another datacube**. The `ndvi` object we creeate in this function uses the bands 8 and 4 from the `datacube` object we pass as argument. It returns a datacube with a single computed band. However, we have the same spatial and time dimmensions!

In [ ]:
def create_ndvi(datacube):
    """
    Calculates Normalized Difference Vegetation Index (NDVI) from a given data cube.

    Args:
        datacube (openeo.DataCube): The data cube containing Red (B04)
            and Near Infra-Red (NIR - B08) bands.

    Returns:
        openeo.DataCube: The computed NDVI data cube
    """
    ndvi = (datacube.band("B08") - datacube.band("B04")) / (
        datacube.band("B08") + datacube.band("B04")
    )
    return ndvi

The following funtion groups previous steps to return a `ndvi` datacube object.

In [ ]:
def get_ndvi(polygon, year):
    """
    Retrieves and processes NDVI data for a given polygon and year.

    Args:
        polygon (dict): The spatial extent of interest in geo-interface format
        year (int or str): The year of interest for NDVI calculation

    Returns:
        openeo.DataCube: Processed NDVI data cube with cloud masking applied
    """
    connection = connect_to_copernicus(
        client_id=os.getenv("COPERNICUS_ID"),
        client_secret=os.getenv("COPERNICUS_SECRET"),
    )

    datacube = create_datacube(polygon, f"{year}-01-01", f"{year}-12-31", connection)
    datacube = mask_clouds(datacube)

    ndvi = create_ndvi(datacube)

    return ndvi

**Reducing by time and aggregating by space are well explained in Open'EO's documentation**.


In time reductions, for each pixel, we "summarize" the NDVI value for all the observation period, in this case we use the median, we could also use maximum values, the mean, or other aggregates. This gives us a 2-D array that represents the median values for the selected period, for each pixel. 

In spatial aggregation, we get the median NDVI value for all the pixels of each image (each 2D array) of the cube. This gives us a time series, with all the individual median values for each observation date.

In [ ]:
def reduce_by_time(ndvi):
    """
    Reduces the temporal dimension of an NDVI data cube by computing the median value.

    Args:
        ndvi (openeo.DataCube): The NDVI data cube with a temporal dimension

    Returns:
        openeo.DataCube: The time-reduced data cube with median values
    """
    ndvi_time_reduced = ndvi.reduce_dimension(dimension="t", reducer="median")
    return ndvi_time_reduced

In [ ]:
def get_time_series(ndvi, polygon):
    """
    Computes a median NDVI time series for the specified polygon geometry.

    Args:
        ndvi (openeo.DataCube): The NDVI data cube with temporal dimension
        polygon (dict): GeoJSON-like polygon geometry for spatial aggregation

    Returns:
        openeo.DataCube: Time series data cube with median NDVI values
    """
    timeseries = ndvi.aggregate_spatial(geometries=polygon, reducer="median")
    return timeseries

Utility functions:


In [ ]:
def download_ndvi(ndvi, park_name, year):
    """
    Downloads an NDVI data cube as a TIFF file, using the dictionary key as the filename.

    Args:
        ndvi: NDVI data cube to be downloaded (xarray DataArray or similar)
        park_name (str): Name of the park/location to include in filename
        year (int/str): Year of the data to include in filename

    Returns:
        int: Always returns 0, indicating successful completion
    """

    ndvi.download(f"{park_name} - {year}.tif")

    return 0

In [ ]:
def download_time_series(ndvi_timeseries, park_name, year):
    title = f"{park_name} - {year}"
    job = ndvi_timeseries.execute_batch(out_format="CSV", title=title)
    job.get_results().download_file(f"{title}.csv")

In [ ]:
def read_time_series(park_name, year):
    time_series_name = f"{park_name} - {year}.csv"
    df_time_series = pd.read_csv(time_series_name, index_col=0)

    df_time_series.index = pd.to_datetime(df_time_series.index)
    df_time_series = df_time_series.rename(columns={"band_unnamed": "ndvi"})
    df_time_series = df_time_series.drop(columns="feature_index")
    df_time_series.sort_index(inplace=True)
    df_time_series = df_time_series.interpolate(method="time")
    df_time_series["date"] = df_time_series.index

    return df_time_series

## Using the functions!

You can use the functions sequentially, see how they work!! Try selecting different parks.

**IMPORTANT:** Processing each image takes sometime, about 5 minutes, especially for the time series!

In [ ]:
polygon = get_polygon(gdf_paris_parks, 1679)

In [ ]:
ndvi = get_ndvi(polygon, 2023)

In [ ]:
reduced_t_ndvi = reduce_by_time(ndvi)

In [ ]:
download_ndvi(reduced_t_ndvi, "Bois de Vincennes", 2023)

In [ ]:
ndvi_time_series = get_time_series(ndvi, polygon)

In [ ]:
download_time_series(ndvi_time_series, "Bois de Vincennes", 2023)

## Usint the resulting objects

In [ ]:
df_time_series = read_time_series("Bois de Vincennes", 2023)

In [ ]:
df_time_series.head()

## Plot images with lotly

Once that we have a `.tif` image, we can use [rasterio](https://pypi.org/project/rasterio/) to open it and return a NumPy array. Take a look at the functions below, we'll use them to plot the result of the selectors in our Taipy app!

In [ ]:
def read_tif(park_name, year):
    """
    Reads a NDVI TIFF file and returns the image data as a numpy array.

    Args:
        park_name (str): Name of the park/location used in the filename
        year (int or str): Year of the data used in the filename

    Returns:
        numpy.ndarray: 2D array containing NDVI values from the TIFF file
    """
    tif_image = f"{park_name} - {year}.tif"

    with rasterio.open(tif_image) as src:
        ndvi_image = src.read(1)  # NDVI is one band

    return ndvi_image

In [ ]:
def plot_ndvi(np_image):
    """
    Create an NDVI image from a NumPy Array.
    Args:
        np_image: np.ndarray: A NumPy array with median season values
            for each pixel.
    Returns:
        Plotly imshow figure
    """
    fig = px.imshow(
        np_image,
        color_continuous_scale="RdYlGn",
        range_color=(-1, 1),
        labels={"color": "NDVI"},
    )
    fig.update_traces(
        hovertemplate="NDVI: %{z}<extra></extra>",  # `%{z}` is the value, `<extra></extra>` removes secondary hover info
    )
    fig.update_layout(title="NDVI Map", coloraxis_colorbar=dict(title="NDVI"))
    return fig

In [ ]:
np_image = read_tif("Bois de Vincennes", 2023)

In [ ]:
plot_ndvi(np_image)

In [ ]:
def plot_box(ndvi_array, park_name, year):
    """
    Creates violin plot of the NDVI median yearly array.

    Args:
    ndvi_array : numpy.ndarray
        Array wuth NDVI values
    park_name : str
        Name of the park
    year : int or str
        Observation year

    Returns:
    violin plot plotly figure
    """
    flattened_data = ndvi_array.flatten()

    fig = px.violin(
        y=flattened_data,
        title=f"NDVI distribution for {park_name}, {year}",
        labels={"y": "NDVI"},
        color_discrete_sequence=["#12b049"],
    )

    fig.update_layout(
        showlegend=False,
        yaxis_title="NDVI Value",
        xaxis_title="",
        plot_bgcolor="rgba(240,240,240,0.8)",
        paper_bgcolor="rgba(240,240,240,0.5)",
        font=dict(family="Arial", size=12),
        margin=dict(l=20, r=20, t=40, b=20),
    )

    # NDVI range for scale: (-1 to 1)
    fig.update_yaxes(range=[-1, 1])

    # Add horizontal lines at important NDVI thresholds
    fig.add_hline(y=0, line_dash="dot", line_color="gray")
    fig.add_hline(y=0.5, line_dash="dot", line_color="gray")
    fig.add_hline(y=-0.5, line_dash="dot", line_color="gray")

    return fig

In [ ]:
plot_box(np_image, "Bois de Vincennes", 2023)

In [ ]:
def plot_ndvi_timeseries(df, title):
    """
    Line plot for time series DataFrame. The plot has indicator lines
    for healthy vegetation and for non-vegetation index rates.

    Args:
        df : DataFrame
            NDVI time series with a 'date' column.
        title : str
            Title for the plot.

    Returns:
        plotly.graph_objects.Figure
            Line chart for the time series.
    """
    df = df.copy()
    df["date"] = pd.to_datetime(
        df["date"]
    ).dt.date  # Convert to date format (remove time)

    title = f"NDVI Trend: {title}"

    fig = px.line(
        df,
        x="date",
        y="ndvi",
        title=title,
        labels={"ndvi": "NDVI Value", "date": "Date"},
        color_discrete_sequence=["#12b049"],
        template="plotly_white",
    )

    fig.update_traces(
        line_width=2.5,
        hovertemplate="<b>Date</b>: %{x|%b %d}<br><b>NDVI</b>: %{y:.2f}",
        mode="lines",
    )

    fig.update_layout(
        yaxis_title="NDVI Value",
        xaxis_title="",
        yaxis_range=[-1, 1.05],
        hovermode="x unified",
        plot_bgcolor="white",
        font=dict(family="Arial"),
        margin=dict(l=50, r=50, t=60, b=30),
    )

    # NDVI reference lines
    for y, color, name in [
        (0.5, "#8BC34A", "Healthy"),
        (0, "#FFC107", "Neutral"),
        (-0.2, "#F44336", "Barren"),
    ]:
        fig.add_hline(
            y=y,
            line_dash="dot",
            line_color=color,
            opacity=0.5,
            annotation_text=name,
            annotation_position="right",
            annotation_font_size=10,
        )

    return fig

In [ ]:
plot_ndvi_timeseries(df_time_series, "Bois de Vincennes")